# AES + HMAC Secure Message (Encrypt-then-MAC)
<ul> 
<li>AES = Hides the content</li>
<li>HMAC = Ensures message hasn't been tampered</li>
<li>This is used in: VPNs, TLS, Smart Grid secure channels</li>
</ul>



In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import hmac
import hashlib

# ---------- Sender Side ----------

print(" -- SENDER SIDE -- ")

# 1. Generate keys
aes_key = get_random_bytes(16)   # AES-128 key
hmac_key = get_random_bytes(32)  # HMAC-SHA256 key

# 2. Create AES cipher with IV
cipher = AES.new(aes_key, AES.MODE_CBC)
iv = cipher.iv

# 3. Encrypt message
plaintext = b"Practicising Encryption"
padded = pad(plaintext, AES.block_size)
ciphertext = cipher.encrypt(padded)

# 4. Generate HMAC over (IV + ciphertext)
msg_to_mac = iv + ciphertext
hmac_digest = hmac.new(hmac_key, msg_to_mac, hashlib.sha256).digest()

# 5. Simulate sending over channel
print("\n📤 SENT OVER CHANNEL:")
print(f"IV           : {iv.hex()}")
print(f"Ciphertext   : {ciphertext.hex()}")
print(f"HMAC Tag     : {hmac_digest.hex()}")

# ---------- Receiver Side ----------

print("\n📥 RECEIVER SIDE")

# 6. Simulate receiving same values
received_iv = iv
received_ciphertext = ciphertext
received_hmac = hmac_digest

print("\n📩 RECEIVED DATA:")
print(f"Received IV         : {received_iv.hex()}")
print(f"Received Ciphertext : {received_ciphertext.hex()}")
print(f"Received HMAC Tag   : {received_hmac.hex()}")

# 7. Verify HMAC
reconstructed_hmac = hmac.new(hmac_key, received_iv + received_ciphertext, hashlib.sha256).digest()

# Constant-time comparison
if hmac.compare_digest(received_hmac, reconstructed_hmac):
    print("\n HMAC Verified. Message is authentic.")

    # 8. Decrypt the message
    decipher = AES.new(aes_key, AES.MODE_CBC, iv=received_iv)
    decrypted_padded = decipher.decrypt(received_ciphertext)
    decrypted = unpad(decrypted_padded, AES.block_size)
    print("🔓 Decrypted Message:", decrypted.decode())

else:
    print("❌ HMAC verification failed. Message is tampered or invalid.")


 -- SENDER SIDE -- 

📤 SENT OVER CHANNEL:
IV           : fafa91e1cc83dcb4de537979176244d8
Ciphertext   : 58c8bdadfff5fe6558fbf61946698fdb7c3e4f8241a67b401df1f739b795f5d2
HMAC Tag     : 5f3201e5188fa1346c23ebc3ae7ddd5b11d993e30c5e3de5a14f980dd19c6487

📥 RECEIVER SIDE

📩 RECEIVED DATA:
Received IV         : fafa91e1cc83dcb4de537979176244d8
Received Ciphertext : 58c8bdadfff5fe6558fbf61946698fdb7c3e4f8241a67b401df1f739b795f5d2
Received HMAC Tag   : 5f3201e5188fa1346c23ebc3ae7ddd5b11d993e30c5e3de5a14f980dd19c6487

 HMAC Verified. Message is authentic.
🔓 Decrypted Message: Practicising Encryption
